# Notebook to test the predictions

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

ML_DIR = Path().resolve().parents[0]
sys.path.append(str(ML_DIR))

# Import ML functions
from update_rolling import update_rolling_stats
from predict import predict_points

load_dotenv()
DATABASE_URL = os.getenv("ML_DATABASE_URL")
engine = create_engine(DATABASE_URL)


In [3]:
df_latest_rolling = update_rolling_stats(engine)

print("Latest rolling rows per player:")
display(df_latest_rolling.head())


No new games to update.
Latest rolling rows per player:


,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers,avg_points_last5,avg_assists_last5,avg_rebounds_last5,avg_minutes_last5
0,2544,22500607,2026-01-18,LAL vs. TOR,LAL,32.0,24.0,7.0,4.0,0.0,2.0,3.0,25.2,6.8,7.0,32.6
1,101108,22500313,2025-12-01,LAC @ MIA,NaN,15.0,8.0,3.0,0.0,0.0,0.0,1.0,3.8,2.4,1.2,13.4
2,200768,22500560,2026-01-12,PHI @ TOR,PHI,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.2,0.6,8.2
3,201142,22500591,2026-01-16,HOU vs. MIN,HOU,40.0,39.0,7.0,4.0,2.0,1.0,5.0,27.8,4.8,7.4,38.8
4,201143,22500584,2026-01-15,GSW vs. NYK,GSW,21.0,5.0,5.0,5.0,1.0,2.0,1.0,6.6,3.8,6.0,18.4


## Run predictions

In [4]:
# df_preds = predict_points(engine)

# print("Predictions:")
# df_preds = df_preds.sort_values("game_date", ascending=False)
# display(df_preds.head(20))


In [6]:
df_preds = predict_points(engine)

# Load players lookup
# df_players = pd.read_sql(
#     "SELECT id AS player_id, full_name FROM players",
#     engine,
# )

# # Merge names
# df_preds = df_preds.merge(
#     df_players,
#     on="player_id",
#     how="left",
# )

print("Predictions:")
df_preds = df_preds.sort_values("pred_points", ascending=False)
display(df_preds.head(20))


C:\Users\kalz9\OneDrive\Documents\onedrive\Documents\Gamblr\gamblr\ml\predict.py:32: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_rolling["game_date"] = pd.to_datetime(df_rolling["game_date"], dayfirst=True)


Predictions:


,player_id,full_name,team_abbreviation,matchup,game_date,pred_points
105,1629029,Luka Dončić,LAL,LAL @ DAL,2026-01-24,34.328892
6,1630178,Tyrese Maxey,PHI,NYK @ PHI,2026-01-24,31.028475
4,203954,Joel Embiid,PHI,NYK @ PHI,2026-01-24,25.656202
51,1642851,Kon Knueppel,CHA,WAS @ CHA,2026-01-24,24.574074
44,1628970,Miles Bridges,CHA,WAS @ CHA,2026-01-24,23.264383
92,203076,Anthony Davis,DAL,LAL @ DAL,2026-01-24,22.888582
68,1631094,Paolo Banchero,ORL,CLE @ ORL,2026-01-24,22.613453
86,1627759,Jaylen Brown,BOS,BOS @ CHI,2026-01-24,21.711468
2,202331,Paul George,PHI,NYK @ PHI,2026-01-24,21.048353
102,2544,LeBron James,LAL,LAL @ DAL,2026-01-24,20.580643
